In [4]:
import pygame
import random

# Constants
WIDTH, HEIGHT = 400, 600
WHITE = (255, 255, 255)
BLACK = (0, 0, 0)
PLATFORM_WIDTH = 70
PLATFORM_HEIGHT = 12
PLATFORM_COLORS = [(0, 200, 0), (0, 0, 255), (255, 0, 0), (255, 255, 0), (128, 0, 128), (255, 165, 0), (255, 192, 203)]

pygame.init()
screen = pygame.display.set_mode((WIDTH, HEIGHT))
clock = pygame.time.Clock()

class Platform(pygame.sprite.Sprite):
    def __init__(self, x, y, color, height):
        super().__init__()
        self.image = pygame.Surface((PLATFORM_WIDTH, PLATFORM_HEIGHT))
        self.image.fill(color)
        self.rect = self.image.get_rect()
        self.rect.x = x
        self.rect.y = y
        self.height = height  # Absolute height value
        
class Player(pygame.sprite.Sprite):
    def __init__(self):
        super().__init__()
        # Create a transparent surface
        self.image = pygame.Surface((30, 50), pygame.SRCALPHA)  # room for head and body

        # Draw head (circle)
        pygame.draw.circle(self.image, BLACK, (15, 10), 8, 100)

        # Draw body (line)
        pygame.draw.line(self.image, BLACK, (15, 18), (15, 40), 4)

        # Draw arms (lines)
        pygame.draw.line(self.image, BLACK, (15, 25), (5, 35), 4)
        pygame.draw.line(self.image, BLACK, (15, 25), (25, 35), 4)

        # Draw legs (lines)
        pygame.draw.line(self.image, BLACK, (15, 40), (5, 50), 4)
        pygame.draw.line(self.image, BLACK, (15, 40), (25, 50), 4)

        self.rect = self.image.get_rect()
        self.rect.center = (WIDTH // 2, HEIGHT // 2)
        self.vel_y = 0

    def update(self):
        # Gravity
        self.vel_y += 0.5
        self.rect.y += int(self.vel_y)
        # Move left/right
        keys = pygame.key.get_pressed()
        if keys[pygame.K_LEFT]:
            self.rect.x -= 5
        if keys[pygame.K_RIGHT]:
            self.rect.x += 5
        # Wrap around
        if self.rect.x > WIDTH:
            self.rect.x = 0
        elif self.rect.x < 0:
            self.rect.x = WIDTH

        # Trigger game over if fallen off
        if self.rect.y > HEIGHT:
            global game_over
            game_over = True

# Set up platforms
platforms = pygame.sprite.Group()
platform_list = []
# Start with a platform at the bottom for the player
start_plat = Platform(WIDTH // 2 - PLATFORM_WIDTH // 2, HEIGHT - 50, PLATFORM_COLORS[0], 0)
platforms.add(start_plat)
platform_list.append(start_plat)
prev_x = start_plat.rect.x
for i in range(1, 7):
    # Ensure platforms are reachable: max gap of 120 pixels
    x = prev_x + random.randint(-120, 120)
    x = max(0, min(WIDTH - PLATFORM_WIDTH, x))
    plat = Platform(x, HEIGHT - 50 - i * 80, PLATFORM_COLORS[i % len(PLATFORM_COLORS)], i * 80)
    platforms.add(plat)
    platform_list.append(plat)
    prev_x = x

player = Player()
# Place player on the starting platform
player.rect.centerx = start_plat.rect.centerx
player.rect.bottom = start_plat.rect.top
player.vel_y = 0
initial_y = HEIGHT - 50  # Player's spawn y
max_height = 0
game_over = False
victory = False
running = True
checkpoint_interval = 500

# Menu state
in_menu = True
start_button_rect = pygame.Rect(WIDTH // 2 - 50, HEIGHT // 2 + 50, 100, 50)

def reset_game():
    global player, platforms, platform_list, max_height, game_over, victory
    platforms.empty()
    platform_list.clear()
    # Regenerate platforms
    start_plat = Platform(WIDTH // 2 - PLATFORM_WIDTH // 2, HEIGHT - 50, PLATFORM_COLORS[0], 0)
    platforms.add(start_plat)
    platform_list.append(start_plat)
    prev_x = start_plat.rect.x
    for i in range(1, 7):
        x = prev_x + random.randint(-120, 120)
        x = max(0, min(WIDTH - PLATFORM_WIDTH, x))
        plat = Platform(x, HEIGHT - 50 - i * 80, PLATFORM_COLORS[i % len(PLATFORM_COLORS)], i * 80)
        platforms.add(plat)
        platform_list.append(plat)
        prev_x = x
    # Place player on the starting platform
    player.rect.centerx = start_plat.rect.centerx
    player.rect.bottom = start_plat.rect.top
    player.vel_y = 0
    max_height = 0
    game_over = False
    victory = False

while running:
    clock.tick(60)
    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            running = False
        elif event.type == pygame.MOUSEBUTTONDOWN and in_menu:
            if start_button_rect.collidepoint(event.pos):
                in_menu = False

    if not in_menu and not victory:
        if not game_over:
            # Update
            player.update()
            # Update max height
            current_height = max(0, initial_y - player.rect.y)
            max_height = max(max_height, current_height)
            # Check for victory
            if max_height >= 5000:
                victory = True
            # Platform collision
            if player.vel_y > 0:
                hits = pygame.sprite.spritecollide(player, platforms, False)
                for hit in hits:
                    if player.rect.bottom <= hit.rect.bottom + 15:
                        player.rect.bottom = hit.rect.top
                        player.vel_y = -12  # Jump velocity

            # Scroll up when player reaches upper part
            if player.rect.top <= HEIGHT // 4:
                scroll_distance = abs(player.vel_y)
                player.rect.y += scroll_distance
                max_height += scroll_distance  # Accumulate max height on scroll
                for plat in platform_list:
                    plat.rect.y += scroll_distance
                    if plat.rect.top >= HEIGHT:
                        # Regenerate with reachable x
                        prev_x = platform_list[-1].rect.x if platform_list else WIDTH // 2
                        x = prev_x + random.randint(-120, 120)
                        x = max(0, min(WIDTH - PLATFORM_WIDTH, x))
                        plat.rect.x = x
                        plat.rect.y = random.randint(-PLATFORM_HEIGHT, 0)
                        prev_x = x

    # Drawing
    screen.fill(WHITE)
    if in_menu:
        # Loading screen
        font_large = pygame.font.Font(None, 72)
        font_small = pygame.font.Font(None, 36)
        title_surf = font_large.render('Jump Game', True, BLACK)
        screen.blit(title_surf, (WIDTH // 2 - title_surf.get_width() // 2, HEIGHT // 2 - 100))
        subtitle_surf = font_small.render('Can you make it to 5000 meters?', True, BLACK)
        screen.blit(subtitle_surf, (WIDTH // 2 - subtitle_surf.get_width() // 2, HEIGHT // 2 - 50))
        # Start button
        pygame.draw.rect(screen, (0, 255, 0), start_button_rect)
        start_text = font_small.render('Start', True, BLACK)
        screen.blit(start_text, (start_button_rect.centerx - start_text.get_width() // 2, start_button_rect.centery - start_text.get_height() // 2))
    elif victory:
        # Victory Screen
        font_large = pygame.font.Font(None, 72)
        font_small = pygame.font.Font(None, 36)
        victory_surf = font_large.render('Victory!', True, BLACK)
        final_height_surf = font_small.render(f'Final Height: {int(max_height)} meters', True, BLACK)
        restart_surf = font_small.render('Press R to Restart or Q to Quit', True, BLACK)
        screen.blit(victory_surf, (WIDTH // 2 - victory_surf.get_width() // 2, HEIGHT // 2 - 100))
        screen.blit(final_height_surf, (WIDTH // 2 - final_height_surf.get_width() // 2, HEIGHT // 2 - 20))
        screen.blit(restart_surf, (WIDTH // 2 - restart_surf.get_width() // 2, HEIGHT // 2 + 40))

        # Handle restart/quit
        keys = pygame.key.get_pressed()
        if keys[pygame.K_r]:
            reset_game()
        elif keys[pygame.K_q]:
            running = False
    elif not game_over:
        platforms.draw(screen)
        screen.blit(player.image, player.rect)
        # Draw Height
        font = pygame.font.Font(None, 36)
        height_surf = font.render(f'Height: {int(max_height)} meters', True, BLACK)
        screen.blit(height_surf, (10, 10))

        # Draw checkpoint line and label at the bottom
        current_checkpoint = (int(max_height) // checkpoint_interval) * checkpoint_interval
        if current_checkpoint > 0:
            y_pos = HEIGHT - 50
            # Draw thick red horizontal line
            pygame.draw.line(screen, (255, 0, 0), (0, y_pos), (WIDTH, y_pos), 4)
            # Draw checkpoint label centered
            checkpoint_text = font.render(f'Checkpoint {current_checkpoint}m', True, (255, 0, 0))
            screen.blit(checkpoint_text, (WIDTH // 2 - checkpoint_text.get_width() // 2, y_pos - 30))

    else:
        # Game Over Screen
        font_large = pygame.font.Font(None, 72)
        font_small = pygame.font.Font(None, 36)
        game_over_surf = font_large.render('Game Over', True, BLACK)
        final_height_surf = font_small.render(f'Final Height: {int(max_height)} meters', True, BLACK)
        restart_surf = font_small.render('Press R to Restart or Q to Quit', True, BLACK)
        screen.blit(game_over_surf, (WIDTH // 2 - game_over_surf.get_width() // 2, HEIGHT // 2 - 100))
        screen.blit(final_height_surf, (WIDTH // 2 - final_height_surf.get_width() // 2, HEIGHT // 2 - 20))
        screen.blit(restart_surf, (WIDTH // 2 - restart_surf.get_width() // 2, HEIGHT // 2 + 40))

        # Handle restart/quit
        keys = pygame.key.get_pressed()
        if keys[pygame.K_r]:
            reset_game()
        elif keys[pygame.K_q]:
            running = False

    pygame.display.flip()

pygame.quit()
